In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import argparse
import sys
import yaml
from annoy import AnnoyIndex

# 전처리

## Basic

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader


def basic_data_load(args):
	"""
	Parameters
	----------
	args.dataset.data_path : str
		데이터 경로를 설정할 수 있는 parser

	Returns
	-------
	data : dict
		학습 및 테스트 데이터가 담긴 사전 형식의 데이터를 반환합니다
	"""

	######################## DATA LOAD
	# users = pd.read_csv(args.dataset.data_path + 'users.csv')
	# books = pd.read_csv(args.dataset.data_path + 'books.csv')
	train_df = pd.read_csv(args.dataset.data_path + 'train_ratings.csv')
	test_df = pd.read_csv(args.dataset.data_path + 'test_ratings.csv')
	sub = pd.read_csv(args.dataset.data_path + 'sample_submission.csv')

	all_df = pd.concat([train_df, test_df], axis=0)

	sparse_cols = ['user_id', 'isbn']

	# 라벨 인코딩하고 인덱스 정보를 저장
	label2idx, idx2label = {}, {}
	for col in sparse_cols:
		all_df[col] = all_df[col].fillna('unknown')
		unique_labels = all_df[col].astype("category").cat.categories
		label2idx[col] = {label: idx for idx, label in enumerate(unique_labels)}
		idx2label[col] = {idx: label for idx, label in enumerate(unique_labels)}
		train_df[col] = train_df[col].map(label2idx[col])
		test_df[col] = test_df[col].map(label2idx[col])

	field_dims = [len(label2idx[col]) for col in sparse_cols]

	# 각 유저별 평점 평균을 구하고 train_df, test_df에 추가
	user_rating_mean = train_df.groupby('user_id')['rating'].mean()
	train_df['user_rating_mean'] = train_df.user_id.map(user_rating_mean)
	test_df['user_rating_mean'] = test_df.user_id.map(user_rating_mean)

	# 각 유저의 평점 개수를 구하고 train_df,  test_df에 추가
	user_rating_count = train_df['user_id'].value_counts()
	train_df['user_rating_count'] = train_df.user_id.map(user_rating_count)
	test_df['user_rating_count'] = test_df.user_id.map(user_rating_count)
	# train_df에 없던 새로운 유저의 경우 평점 개수를 0으로 설정
	test_df['user_rating_count'] = test_df['user_rating_count'].fillna(0)

	# isbn 별 평점 평균을 구하고 train_df, test_df에 추가
	book_rating_mean = train_df.groupby('isbn')['rating'].mean()
	train_df['book_rating_mean'] = train_df.isbn.map(book_rating_mean)
	test_df['book_rating_mean'] = test_df.isbn.map(book_rating_mean)

	# isbn 별 평점 평균과의 차이를 구하고 train_df에 추가
	train_df['diff_book_rating_mean'] = train_df['rating'] - train_df['book_rating_mean']

	data = {
		'train': train_df,
		'test': test_df.drop(['rating'], axis=1),
		'field_dims': field_dims,
		'label2idx': label2idx,
		'idx2label': idx2label,
		'sub': sub,
	}

	return data


def basic_data_split(args, data):
	"""
	Parameters
	----------
	args.dataset.valid_ratio : float
		Train/Valid split 비율을 입력합니다.
	args.seed : int
		데이터 셔플 시 사용할 seed 값을 입력합니다.

	Returns
	-------
	data : dict
		data 내의 학습 데이터를 학습/검증 데이터로 나누어 추가한 후 반환합니다.
	"""
	if args.dataset.valid_ratio == 0:
		data['X_train'] = data['train'].drop('rating', axis=1)
		data['y_train'] = data['train']['rating']

	else:
		X_train, X_valid, y_train, y_valid = train_test_split(
			data['train'].drop(['rating'], axis=1),
			data['train']['rating'],
			test_size=args.dataset.valid_ratio,
			random_state=args.seed,
			shuffle=True
		)
		data['X_train'], data['X_valid'], data['y_train'], data['y_valid'] = X_train, X_valid, y_train, y_valid

	return data


def basic_data_loader(args, data):
	"""
	Parameters
	----------
	args.dataloader.batch_size : int
		데이터 batch에 사용할 데이터 사이즈
	args.dataloader.shuffle : bool
		data shuffle 여부
	args.dataloader.num_workers: int
		dataloader에서 사용할 멀티프로세서 수
	args.dataset.valid_ratio : float
		Train/Valid split 비율로, 0일 경우에 대한 처리를 위해 사용합니다.
	data : dict
		basic_data_split 함수에서 반환된 데이터

	Returns
	-------
	data : dict
		DataLoader가 추가된 데이터를 반환합니다.
	"""

	train_dataset = TensorDataset(torch.LongTensor(data['X_train'].values), torch.LongTensor(data['y_train'].values))
	valid_dataset = TensorDataset(torch.LongTensor(data['X_valid'].values),
								  torch.LongTensor(data['y_valid'].values)) if args.dataset.valid_ratio != 0 else None
	test_dataset = TensorDataset(torch.LongTensor(data['test'].values))

	train_dataloader = DataLoader(train_dataset, batch_size=args.dataloader.batch_size, shuffle=args.dataloader.shuffle,
								  num_workers=args.dataloader.num_workers)
	valid_dataloader = DataLoader(valid_dataset, batch_size=args.dataloader.batch_size, shuffle=False,
								  num_workers=args.dataloader.num_workers) if args.dataset.valid_ratio != 0 else None
	test_dataloader = DataLoader(test_dataset, batch_size=args.dataloader.batch_size, shuffle=False,
								 num_workers=args.dataloader.num_workers)

	data['train_dataloader'], data['valid_dataloader'], data[
		'test_dataloader'] = train_dataloader, valid_dataloader, test_dataloader

	return data


## Context

In [3]:
import numpy as np
import pandas as pd
import regex
import torch
from torch.utils.data import TensorDataset, DataLoader


def str2list(x: str) -> list:
	'''문자열을 리스트로 변환하는 함수'''
	return x[1:-1].split(', ')


def split_location(x: str) -> list:
	'''
	Parameters
	----------
	x : str
		location 데이터

	Returns
	-------
	res : list
		location 데이터를 나눈 뒤, 정제한 결과를 반환합니다.
		순서는 country, state, city, ... 입니다.
	'''
	res = x.split(',')
	res = [i.strip().lower() for i in res]
	res = [regex.sub(r'[^a-zA-Z/ ]', '', i) for i in res]  # remove special characters
	res = [i if i not in ['n/a', ''] else np.nan for i in res]  # change 'n/a' into NaN
	res.reverse()  # reverse the list to get country, state, city, ... order

	for i in range(len(res) - 1, 0, -1):
		if (res[i] in res[:i]) and (not pd.isna(res[i])):  # remove duplicated values if not NaN
			res.pop(i)

	return res

def process_context_data(users, books):
	"""
	Parameters
	----------
	users : pd.DataFrame
		users.csv를 인덱싱한 데이터
	books : pd.DataFrame
		books.csv를 인덱싱한 데이터
	ratings1 : pd.DataFrame
		train 데이터의 rating
	ratings2 : pd.DataFrame
		test 데이터의 rating

	Returns
	-------
	label_to_idx : dict
		데이터를 인덱싱한 정보를 담은 딕셔너리
	idx_to_label : dict
		인덱스를 다시 원래 데이터로 변환하는 정보를 담은 딕셔너리
	train_df : pd.DataFrame
		train 데이터
	test_df : pd.DataFrame
		test 데이터
	"""

	users_ = users.copy()
	books_ = books.copy()

	# 데이터 전처리 (전처리는 각자의 상황에 맞게 진행해주세요!)
	books_['category'] = books_['category'].apply(lambda x: str2list(x)[0] if not pd.isna(x) else np.nan)
	books_['language'] = books_['language'].fillna(books_['language'].mode()[0])
	books_['publication_range'] = books_['year_of_publication'].apply(
		lambda x: x // 10 * 10)  # 1990년대, 2000년대, 2010년대, ...

	users_['age'] = users_['age'].fillna(users_['age'].mode()[0])
	users_['age_range'] = users_['age'].apply(lambda x: x // 10 * 10)  # 10대, 20대, 30대, ...

	users_['location_list'] = users_['location'].apply(lambda x: split_location(x))
	users_['location_country'] = users_['location_list'].apply(lambda x: x[0])
	users_['location_state'] = users_['location_list'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
	users_['location_city'] = users_['location_list'].apply(lambda x: x[2] if len(x) > 2 else np.nan)
	for idx, row in users_.iterrows():
		if (not pd.isna(row['location_state'])) and pd.isna(row['location_country']):
			fill_country = users_[users_['location_state'] == row['location_state']]['location_country'].mode()
			fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
			users_.loc[idx, 'location_country'] = fill_country
		elif (not pd.isna(row['location_city'])) and pd.isna(row['location_state']):
			if not pd.isna(row['location_country']):
				fill_state = users_[(users_['location_country'] == row['location_country'])
									& (users_['location_city'] == row['location_city'])]['location_state'].mode()
				fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
				users_.loc[idx, 'location_state'] = fill_state
			else:
				fill_state = users_[users_['location_city'] == row['location_city']]['location_state'].mode()
				fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
				fill_country = users_[users_['location_city'] == row['location_city']]['location_country'].mode()
				fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
				users_.loc[idx, 'location_country'] = fill_country
				users_.loc[idx, 'location_state'] = fill_state

	users_ = users_.drop(['location'], axis=1)

	return users_, books_


def context_data_load(args):
	"""
	Parameters
	----------
	args.dataset.data_path : str
		데이터 경로를 설정할 수 있는 parser

	Returns
	-------
	data : dict
		학습 및 테스트 데이터가 담긴 사전 형식의 데이터를 반환합니다.
	"""

	######################## DATA LOAD
	users = pd.read_csv(args.dataset.data_path + 'users.csv')
	books = pd.read_csv(args.dataset.data_path + 'books.csv')
	train = pd.read_csv(args.dataset.data_path + 'train_ratings.csv')
	test = pd.read_csv(args.dataset.data_path + 'test_ratings.csv')
	sub = pd.read_csv(args.dataset.data_path + 'sample_submission.csv')

	users_, books_ = process_context_data(users, books)

	# 유저 및 책 정보를 합쳐서 데이터 프레임 생성
	# 사용할 컬럼을 user_features와 book_features에 정의합니다. (단, 모두 범주형 데이터로 가정)
	# 베이스라인에서는 가능한 모든 컬럼을 사용하도록 구성하였습니다.
	# NCF를 사용할 경우, idx 0, 1은 각각 user_id, isbn이어야 합니다.
	user_features = ['user_id', 'age_range', 'location_country', 'location_state', 'location_city']
	book_features = ['isbn', 'book_title', 'book_author', 'publisher', 'language', 'category', 'publication_range']
	sparse_cols = ['user_id', 'isbn'] + list(
		set(user_features + book_features) - {'user_id', 'isbn'}) if args.model == 'NCF' \
		else user_features + book_features

	# 선택한 컬럼만 추출하여 데이터 조인
	train_df = train.merge(users_, on='user_id', how='left') \
		.merge(books_, on='isbn', how='left')[sparse_cols + ['rating']]
	test_df = test.merge(users_, on='user_id', how='left') \
		.merge(books_, on='isbn', how='left')[sparse_cols]
	all_df = pd.concat([train_df, test_df], axis=0)

	# feature_cols의 데이터만 라벨 인코딩하고 인덱스 정보를 저장
	label2idx, idx2label = {}, {}
	for col in sparse_cols:
		all_df[col] = all_df[col].fillna('unknown')
		unique_labels = all_df[col].astype("category").cat.categories
		label2idx[col] = {label: idx for idx, label in enumerate(unique_labels)}
		idx2label[col] = {idx: label for idx, label in enumerate(unique_labels)}
		train_df[col] = train_df[col].map(label2idx[col])
		test_df[col] = test_df[col].map(label2idx[col])

	field_dims = [len(label2idx[col]) for col in train_df.columns if col != 'rating']

	data = {
		'train': train_df,
		'test': test_df,
		'field_names': sparse_cols,
		'field_dims': field_dims,
		'label2idx': label2idx,
		'idx2label': idx2label,
		'sub': sub,
	}

	return data


def context_data_split(args, data):
	'''data 내의 학습 데이터를 학습/검증 데이터로 나누어 추가한 후 반환합니다.'''
	return basic_data_split(args, data)


def context_data_loader(args, data):
	"""
	Parameters
	----------
	args.dataloader.batch_size : int
		데이터 batch에 사용할 데이터 사이즈
	args.dataloader.shuffle : bool
		data shuffle 여부
	args.dataloader.num_workers: int
		dataloader에서 사용할 멀티프로세서 수
	args.dataset.valid_ratio : float
		Train/Valid split 비율로, 0일 경우에 대한 처리를 위해 사용합니다.
	data : dict
		context_data_load 함수에서 반환된 데이터

	Returns
	-------
	data : dict
		DataLoader가 추가된 데이터를 반환합니다.
	"""

	train_dataset = TensorDataset(torch.LongTensor(data['X_train'].values), torch.LongTensor(data['y_train'].values))
	valid_dataset = TensorDataset(torch.LongTensor(data['X_valid'].values),
								  torch.LongTensor(data['y_valid'].values)) if args.dataset.valid_ratio != 0 else None
	test_dataset = TensorDataset(torch.LongTensor(data['test'].values))

	train_dataloader = DataLoader(train_dataset, batch_size=args.dataloader.batch_size, shuffle=args.dataloader.shuffle,
								  num_workers=args.dataloader.num_workers)
	valid_dataloader = DataLoader(valid_dataset, batch_size=args.dataloader.batch_size, shuffle=False,
								  num_workers=args.dataloader.num_workers) if args.dataset.valid_ratio != 0 else None
	test_dataloader = DataLoader(test_dataset, batch_size=args.dataloader.batch_size, shuffle=False,
								 num_workers=args.dataloader.num_workers)

	data['train_dataloader'], data['valid_dataloader'], data[
		'test_dataloader'] = train_dataloader, valid_dataloader, test_dataloader

	return data


## text

In [4]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel

def text_preprocessing(summary):
    """
    Parameters
    ----------
    summary : pd.Series
        정규화와 같은 기본적인 전처리를 하기 위한 텍스트 데이터를 입력합니다.
    
    Returns
    -------
    summary : pd.Series
        전처리된 텍스트 데이터를 반환합니다.
        베이스라인에서는 특수문자 제거, 공백 제거를 진행합니다.
    """
    summary = re.sub("[^0-9a-zA-Z.,!?]", " ", summary)  # .,!?를 제외한 특수문자 제거
    summary = re.sub("\s+", " ", summary)  # 중복 공백 제거

    return summary


def text_to_vector(text, tokenizer, model):
    """
    Parameters
    ----------
    text : str
        `summary_merge()`를 통해 병합된 요약 데이터
    tokenizer : Tokenizer
        텍스트 데이터를 `model`에 입력하기 위한 토크나이저
    model : 사전학습된 언어 모델
        텍스트 데이터를 벡터로 임베딩하기 위한 모델
    ----------
    """
    text_ = "[CLS] " + text + " [SEP]"
    tokenized = tokenizer.encode(text_, add_special_tokens=True)
    token_tensor = torch.tensor([tokenized], device=model.device)
    with torch.no_grad():
        outputs = model(token_tensor)  # attention_mask를 사용하지 않아도 됨
        ### BERT 모델의 경우, 최종 출력물의 사이즈가 (토큰길이, 임베딩=768)이므로, 이를 평균내어 사용하거나 pooler_output을 사용하여 [CLS] 토큰의 임베딩만 사용
        # sentence_embedding = torch.mean(outputs.last_hidden_state[0], dim=0)  # 방법1) 모든 토큰의 임베딩을 평균내어 사용
        sentence_embedding = outputs.pooler_output.squeeze(0)  # 방법2) pooler_output을 사용하여 맨 첫 토큰인 [CLS] 토큰의 임베딩만 사용
    
    return sentence_embedding.cpu().detach().numpy() 


def process_text_data(ratings, users, books, tokenizer, model, vector_create=False):
    """
    Parameters
    ----------
    users : pd.DataFrame
        유저 정보에 대한 데이터 프레임을 입력합니다.
    books : pd.DataFrame
        책 정보에 대한 데이터 프레임을 입력합니다.
    vector_create : bool
        사전에 텍스트 데이터 벡터화가 된 파일이 있는지 여부를 입력합니다.

    Returns
    -------
    `users_` : pd.DataFrame
        각 유저가 읽은 책에 대한 요약 정보를 병합 및 벡터화하여 추가한 데이터 프레임을 반환합니다.

    `books_` : pd.DataFrame
        텍스트 데이터를 벡터화하여 추가한 데이터 프레임을 반환합니다.
    """
    num2txt = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five']
    users_ = users.copy()
    books_ = books.copy()
    nan_value = 'None'
    books_['summary'] = books_['summary'].fillna(nan_value)\
                                         .apply(lambda x: text_preprocessing(x))\
                                         .replace({'': nan_value, ' ': nan_value})
    
    books_['summary_length'] = books_['summary'].apply(lambda x:len(x))
    books_['review_count'] = books_['isbn'].map(ratings['isbn'].value_counts())

    users_['books_read'] = users_['user_id'].map(ratings.groupby('user_id')['isbn'].apply(list))

    if vector_create:
        if not os.path.exists('./data/text_vector'):
            os.makedirs('./data/text_vector')

        print('Create Item Summary Vector')
        book_summary_vector_list = []
        for title, summary in tqdm(zip(books_['book_title'], books_['summary']), total=len(books_)):
            # 책에 대한 텍스트 프롬프트는 아래와 같이 구성됨
            # '''
            # Book Title: {title}
            # Summary: {summary}
            # '''
            prompt_ = f'Book Title: {title}\n Summary: {summary}\n'
            vector = text_to_vector(prompt_, tokenizer, model)
            book_summary_vector_list.append(vector)
        
        book_summary_vector_list = np.concatenate([
                                                books_['isbn'].values.reshape(-1, 1),
                                                np.asarray(book_summary_vector_list, dtype=np.float32)
                                                ], axis=1)
        
        np.save('./data/text_vector/book_summary_vector.npy', book_summary_vector_list)

        print('Create User Summary Merge Vector')

        user_summary_merge_vector_list = []
        for books_read in tqdm(users_['books_read']):
            if not isinstance(books_read, list) and pd.isna(books_read):  # 유저가 읽은 책이 없는 경우, 텍스트 임베딩을 0으로 처리
                user_summary_merge_vector_list.append(np.zeros((768)))
                continue
            
            read_books = books_[books_['isbn'].isin(books_read)][['book_title', 'summary', 'review_count']]
            read_books = read_books.sort_values('review_count', ascending=False).head(5)  # review_count가 높은 순으로 5개의 책을 선택
            # 유저에 대한 텍스트 프롬프트는 아래와 같이 구성됨
            # DeepCoNN에서 유저의 리뷰를 요약하여 하나의 벡터로 만들어 사용함을 참고 (https://arxiv.org/abs/1701.04783)
            # '''
            # Five Books That You Read
            # 1. Book Title: {title}
            # Summary: {summary}
            # ...
            # 5. Book Title: {title}
            # Summary: {summary}
            # '''
            prompt_ = f'{num2txt[len(read_books)]} Books That You Read\n'
            for idx, (title, summary) in enumerate(zip(read_books['book_title'], read_books['summary'])):
                summary = summary if len(summary) < 100 else f'{summary[:100]} ...'
                prompt_ += f'{idx+1}. Book Title: {title}\n Summary: {summary}\n'
            vector = text_to_vector(prompt_, tokenizer, model)
            user_summary_merge_vector_list.append(vector)
        
        user_summary_merge_vector_list = np.concatenate([
                                                         users_['user_id'].values.reshape(-1, 1),
                                                         np.asarray(user_summary_merge_vector_list, dtype=np.float32)
                                                        ], axis=1)
        
        np.save('./data/text_vector/user_summary_merge_vector.npy', user_summary_merge_vector_list)        
        
    else:
        print('Check Vectorizer')
        print('Vector Load')
        book_summary_vector_list = np.load('./data/text_vector/book_summary_vector.npy', allow_pickle=True)
        user_summary_merge_vector_list = np.load('./data/text_vector/user_summary_merge_vector.npy', allow_pickle=True)

    book_summary_vector_df = pd.DataFrame({'isbn': book_summary_vector_list[:, 0]})
    book_summary_vector_df['book_summary_vector'] = list(book_summary_vector_list[:, 1:].astype(np.float32))
    user_summary_vector_df = pd.DataFrame({'user_id': user_summary_merge_vector_list[:, 0]})
    user_summary_vector_df['user_summary_merge_vector'] = list(user_summary_merge_vector_list[:, 1:].astype(np.float32))

    books_ = pd.merge(books_, book_summary_vector_df, on='isbn', how='left')
    users_ = pd.merge(users_, user_summary_vector_df, on='user_id', how='left')

    return users_, books_


class Text_Dataset(Dataset):
    def __init__(self, user_book_vector, user_summary_vector, book_summary_vector, rating=None):
        """
        Parameters
        ----------
        user_book_vector : np.ndarray
            벡터화된 유저와 책 데이터를 입렵합니다.
        user_summary_vector : np.ndarray
            벡터화된 유저에 대한 요약 정보 데이터를 입력합니다.
        book_summary_vector : np.ndarray
            벡터화된 책에 대한 요약 정보 데이터 입력합니다.
        label : np.ndarray
            정답 데이터를 입력합니다.
        ----------
        """
        self.user_book_vector = user_book_vector
        self.user_summary_vector = user_summary_vector
        self.book_summary_vector = book_summary_vector
        self.rating = rating

    def __len__(self):
        return self.user_book_vector.shape[0]

    def __getitem__(self, i):
        return {
                'user_book_vector' : torch.tensor(self.user_book_vector[i], dtype=torch.long),
                'user_summary_vector' : torch.tensor(self.user_summary_vector[i], dtype=torch.float32),
                'book_summary_vector' : torch.tensor(self.book_summary_vector[i], dtype=torch.float32),
                'rating' : torch.tensor(self.rating[i], dtype=torch.float32),
                } if self.rating is not None else \
                {
                'user_book_vector' : torch.tensor(self.user_book_vector[i], dtype=torch.long),
                'user_summary_vector' : torch.tensor(self.user_summary_vector[i], dtype=torch.float32),
                'book_summary_vector' : torch.tensor(self.book_summary_vector[i], dtype=torch.float32),
                }


def text_data_load(args):
    """
    Parameters
    ----------
    args.dataset.data_path : str
        데이터 경로를 설정할 수 있는 parser
    args.model_args[args.model].pretrained_model : str
        사전학습된 모델을 설정할 수 있는 parser
    args.model_args[args.model].vector_create : bool
        텍스트 데이터 벡터화 및 저장 여부를 설정할 수 있는 parser
        False로 설정하면 기존에 저장된 벡터를 불러옵니다.

    Returns
    -------
    data : dict
        학습 및 테스트 데이터가 담긴 사전 형식의 데이터를 반환합니다.
    """
    users = pd.read_csv(args.dataset.data_path + 'users.csv')
    books = pd.read_csv(args.dataset.data_path + 'books.csv')
    train = pd.read_csv(args.dataset.data_path + 'train_ratings.csv')
    test = pd.read_csv(args.dataset.data_path + 'test_ratings.csv')
    sub = pd.read_csv(args.dataset.data_path + 'sample_submission.csv')

    tokenizer = AutoTokenizer.from_pretrained(args.model_args[args.model].pretrained_model)
    model = AutoModel.from_pretrained(args.model_args[args.model].pretrained_model).to(device=args.device)
    model.eval()
    users_, books_ = process_text_data(train, users, books, tokenizer, model, args.model_args[args.model].vector_create)

    # 유저 및 책 정보를 합쳐서 데이터 프레임 생성 (단, 베이스라인에서는 user_id, isbn, user_summary_merge_vector, book_summary_vector만 사용함)
    # 사용할 컬럼을 user_features와 book_features에 정의합니다. (단, 모두 범주형 데이터로 가정)
    user_features = []
    book_features = []
    sparse_cols = ['user_id', 'isbn'] + list(set(user_features + book_features) - {'user_id', 'isbn'})
    
    train_df = train.merge(books_, on='isbn', how='left')\
                    .merge(users_, on='user_id', how='left')[sparse_cols + ['user_summary_merge_vector', 'book_summary_vector', 'rating']]
    test_df = test.merge(books_, on='isbn', how='left')\
                  .merge(users_, on='user_id', how='left')[sparse_cols + ['user_summary_merge_vector', 'book_summary_vector']]
    all_df = pd.concat([train, test], axis=0)

    # feature_cols의 데이터만 라벨 인코딩하고 인덱스 정보를 저장
    label2idx, idx2label = {}, {}
    for col in sparse_cols:
        all_df[col] = all_df[col].fillna('unknown')
        unique_labels = all_df[col].astype("category").cat.categories
        label2idx[col] = {label:idx for idx, label in enumerate(unique_labels)}
        idx2label[col] = {idx:label for idx, label in enumerate(unique_labels)}
        train_df[col] = train_df[col].map(label2idx[col])
        test_df[col] = test_df[col].map(label2idx[col])

    field_dims = [len(label2idx[col]) for col in sparse_cols]

    data = {
            'train':train_df,
            'test':test_df,
            'field_names':sparse_cols,
            'field_dims':field_dims,
            'label2idx':label2idx,
            'idx2label':idx2label,
            'sub':sub,
            }
    
    return data


def text_data_split(args, data):
    """학습 데이터를 학습/검증 데이터로 나누어 추가한 후 반환합니다."""
    return basic_data_split(args, data)


def text_data_loader(args, data):
    """
    Parameters
    ----------
    args.dataloader.batch_size : int
        데이터 batch에 사용할 데이터 사이즈
    args.dataloader.shuffle : bool
        data shuffle 여부
    args.dataloader.num_workers: int
        dataloader에서 사용할 멀티프로세서 수
    args.dataset.valid_ratio : float
        Train/Valid split 비율로, 0일 경우에 대한 처리를 위해 사용
    data : dict
        text_data_load()에서 반환된 데이터

    Returns
    -------
    data : dict
        Text_Dataset 형태의 학습/검증/테스트 데이터를 DataLoader로 변환하여 추가한 후 반환합니다.
    """
    train_dataset = Text_Dataset(
                                data['X_train'][data['field_names']].values,
                                data['X_train']['user_summary_merge_vector'].values,
                                data['X_train']['book_summary_vector'].values,
                                data['y_train'].values
                                )
    valid_dataset = Text_Dataset(
                                data['X_valid'][data['field_names']].values,
                                data['X_valid']['user_summary_merge_vector'].values,
                                data['X_valid']['book_summary_vector'].values,
                                data['y_valid'].values
                                ) if args.dataset.valid_ratio != 0 else None
    test_dataset = Text_Dataset(
                                data['test'][data['field_names']].values,
                                data['test']['user_summary_merge_vector'].values,
                                data['test']['book_summary_vector'].values,
                                )


    train_dataloader = DataLoader(train_dataset, batch_size=args.dataloader.batch_size, shuffle=args.dataloader.shuffle, num_workers=args.dataloader.num_workers)
    valid_dataloader = DataLoader(valid_dataset, batch_size=args.dataloader.batch_size, shuffle=False, num_workers=args.dataloader.num_workers) if args.dataset.valid_ratio != 0 else None
    test_dataloader = DataLoader(test_dataset, batch_size=args.dataloader.batch_size, shuffle=False, num_workers=args.dataloader.num_workers)
    data['train_dataloader'], data['valid_dataloader'], data['test_dataloader'] = train_dataloader, valid_dataloader, test_dataloader
    
    return data


/opt/conda/envs/boostcamp11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## config

In [5]:
from omegaconf import OmegaConf

# YAML 파일을 불러오는 함수
def load_config(file_path):
    config = OmegaConf.load(file_path)
    return config

# config.yaml 파일을 불러와서 계층형으로 접근할 수 있게 함
config = load_config('./config/config_baseline.yaml')

# 예제 출력
print(f'dataset.data_path: {config.dataset.data_path}')
print(f'dataset.valid_ratio: {config.dataset.valid_ratio}')
print(f'dataloader.batch_size: {config.dataloader.batch_size}')
print(f'optimizer.type: {config.optimizer.type}')
print(f'model_args.FM.embed_dim: {config.model_args.FM.embed_dim}')

config.model = 'Text_DeepFM'
config.model_args[config.model].vector_create = False

dataset.data_path: data/
dataset.valid_ratio: 0.2
dataloader.batch_size: 1024
optimizer.type: Adam
model_args.FM.embed_dim: 16


## data load

In [6]:
data = text_data_load(config)

Check Vectorizer
Vector Load


In [7]:
data = text_data_split(config, data)

In [10]:
X_train = data['X_train']
y_train = data['y_train']
X_valid = data['X_valid']
y_valid = data['y_valid']
X_test = data['test']

In [11]:
X_for_train = pd.concat([X_train, X_valid], axis=0)
y_for_train = pd.concat([y_train, y_valid], axis=0)

In [12]:
# X_train과 y_train을 병합
train_df = X_train.copy()
train_df['rating'] = y_train

# 동일한 isbn을 가진 데이터의 rating 평균을 구하여 저장
isbn_rating_mean = train_df.groupby('isbn')['rating'].mean()
X_train['isbn_rating_mean'] = X_train['isbn'].map(isbn_rating_mean)

# 동일한 isbn을 가진 데이터의 rating 평균과의 차이를 구g함
isbn_diff = y_train - X_train['isbn_rating_mean'] 

In [13]:
# X_train과 y_train을 병합
train_df = X_for_train.copy()
train_df['rating'] = y_for_train

# 동일한 isbn을 가진 데이터의 rating 평균을 구하여 저장
isbn_rating_mean = train_df.groupby('isbn')['rating'].mean()
X_for_train['isbn_rating_mean'] = X_for_train['isbn'].map(isbn_rating_mean)

# 동일한 isbn을 가진 데이터의 rating 평균과의 차이를 구g함
isbn_diff_for_train = y_for_train - X_for_train['isbn_rating_mean'] 

In [14]:
# y_train의 값을 isbn_diff로 대체
y_train_diff = isbn_diff

In [15]:
y_diff_for_train = isbn_diff_for_train

In [25]:
# Annoy 인덱스 생성
vector_length = len(X_train['book_summary_vector'].iloc[0])
annoy_index = AnnoyIndex(vector_length, 'angular')

# X_train의 book_summary_vector를 Annoy 인덱스에 추가
for i, vector in enumerate(X_train['book_summary_vector']):
    annoy_index.add_item(i, vector)

# Annoy 인덱스 빌드
annoy_index.build(10)  # 트리의 개수

# X_valid에서만 존재하는 isbn에 대한 isbn_rating_mean을 대체
def find_nearest_isbn_rating_mean(row):
    if row['isbn'] in X_train['isbn'].values:
        return X_train.loc[X_train['isbn'] == row['isbn'], 'isbn_rating_mean'].values[0]
    else:
        vector = row['book_summary_vector']
        nearest_index = annoy_index.get_nns_by_vector(vector, 1)[0]
        return X_train.iloc[nearest_index]['isbn_rating_mean']

X_valid['isbn_rating_mean'] = X_valid.apply(find_nearest_isbn_rating_mean, axis=1)

# 결과 출력
print(X_valid)

        user_id    isbn                          user_summary_merge_vector  \
72064     64583   25962  [-0.8408327, -0.4360084, -0.90918756, 0.760248...   
165810    43626   54250  [-0.84706074, -0.39163983, -0.8053944, 0.6616,...   
268428    32495  118527  [-0.8577756, -0.4652395, -0.92259413, 0.695843...   
41373     24096   29368  [-0.8388007, -0.42792618, -0.81633306, 0.69966...   
172673    54540   18616  [-0.5625498, -0.22993332, -0.8351104, 0.334444...   
...         ...     ...                                                ...   
296689    53998  103937  [-0.74025136, -0.47905624, -0.9157893, 0.50602...   
120752    32937     651  [-0.75517845, -0.5544599, -0.9751944, 0.714054...   
118345    62156   63251  [-0.71679574, -0.36738247, -0.87868947, 0.5782...   
43645     13765   24492  [-0.8896811, -0.5306489, -0.9459932, 0.6995699...   
202960    44183   77051  [-0.43876028, -0.27600208, -0.884492, 0.275648...   

                                      book_summary_vector  isbn

In [16]:
# Annoy 인덱스 생성
vector_length = len(X_for_train['book_summary_vector'].iloc[0])
annoy_index = AnnoyIndex(vector_length, 'angular')

# X_for_train의 book_summary_vector를 Annoy 인덱스에 추가
for i, vector in enumerate(X_for_train['book_summary_vector']):
    annoy_index.add_item(i, vector)

# Annoy 인덱스 빌드
annoy_index.build(10)  # 트리의 개수

# X_test에서만 존재하는 isbn에 대한 isbn_rating_mean을 대체
def find_nearest_isbn_rating_mean(row):
    if row['isbn'] in X_for_train['isbn'].values:
        return X_for_train.loc[X_for_train['isbn'] == row['isbn'], 'isbn_rating_mean'].values[0]
    else:
        vector = row['book_summary_vector']
        nearest_index = annoy_index.get_nns_by_vector(vector, 1)[0]
        return X_for_train.iloc[nearest_index]['isbn_rating_mean']

X_test['isbn_rating_mean'] = X_test.apply(find_nearest_isbn_rating_mean, axis=1)

# 결과 출력
print(X_test)

       user_id    isbn                          user_summary_merge_vector  \
0         2719      39  [-0.74905014, -0.37544724, -0.58153605, 0.5005...   
1        28656      39  [-0.26531896, -0.40486306, -0.97681594, 0.3710...   
2        37434    4667  [-0.54214895, -0.4519513, -0.98219, 0.4661231,...   
3        38564   32059  [-0.83539337, -0.3439815, -0.7150209, 0.616658...   
4        16742   42311  [-0.7145952, -0.35694596, -0.9835796, 0.582363...   
...        ...     ...                                                ...   
76694    67993  126235  [-0.74315584, -0.4225567, -0.95433205, 0.62949...   
76695    68001  141816  [-0.8367616, -0.5226269, -0.9474142, 0.7494029...   
76696    68013  133571  [-0.76280177, -0.49358055, -0.93778366, 0.7140...   
76697    68024  145168  [-0.83159566, -0.51806724, -0.9570092, 0.73350...   
76698    68066   91703  [-0.60563767, -0.40665743, -0.72936785, 0.3261...   

                                     book_summary_vector  isbn_rating_mean 

In [20]:
X_train_mean = X_train['isbn_rating_mean']
X_valid_mean = X_valid['isbn_rating_mean']
X_train = X_train.drop(['isbn_rating_mean'], axis=1)
X_valid = X_valid.drop(['isbn_rating_mean'], axis=1)

KeyError: 'isbn_rating_mean'

In [19]:
X_for_train_mean = X_for_train['isbn_rating_mean']
X_test_mean = X_test['isbn_rating_mean']
X_for_train = X_for_train.drop(['isbn_rating_mean'], axis=1)
X_test = X_test.drop(['isbn_rating_mean'], axis=1)

In [35]:
y_valid_diff = y_valid - X_valid_mean

In [29]:
X_train = X_train.drop(['user_summary_merge_vector', 'book_summary_vector'], axis=1)
X_valid = X_valid.drop(['user_summary_merge_vector', 'book_summary_vector'], axis=1)

In [21]:
X_for_train = X_for_train.drop(['user_summary_merge_vector', 'book_summary_vector'], axis=1)
X_test = X_test.drop(['user_summary_merge_vector', 'book_summary_vector'], axis=1)

In [37]:
train_df = pd.concat([X_train, y_train_diff], axis=1)
valid_df = pd.concat([X_valid, y_valid_diff], axis=1)

In [22]:
df_for_train = pd.concat([X_for_train, y_diff_for_train], axis=1)

In [39]:
# 마지막 열의 이름을 rating으로 변경
train_df.columns = list(train_df.columns[:-1]) + ['rating']
valid_df.columns = list(valid_df.columns[:-1]) + ['rating']

In [23]:
df_for_train.columns = list(df_for_train.columns[:-1]) + ['rating']

In [26]:
X_test

,user_id,isbn
0,2719,39
1,28656,39
2,37434,4667
3,38564,32059
4,16742,42311
...,...,...
76694,67993,126235
76695,68001,141816
76696,68013,133571
76697,68024,145168


In [27]:
X_test_mean

0        6.857143
1        6.857143
2        8.000000
3        7.600000
4        7.571429
           ...   
76694    8.000000
76695    7.500000
76696    9.000000
76697    6.000000
76698    7.750000
Name: isbn_rating_mean, Length: 76699, dtype: float64

In [65]:
class RatingsDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_id'].values, dtype=torch.long)
        self.items = torch.tensor(df['isbn'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

train_dataset = RatingsDataset(train_df)
valid_dataset = RatingsDataset(valid_df)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [28]:
class RatingsDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_id'].values, dtype=torch.long)
        self.items = torch.tensor(df['isbn'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

train_dataset = RatingsDataset(df_for_train)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [29]:
import torch.nn as nn

class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Bias terms
        self.user_bias = nn.Embedding(num_users, 1)
        
    def forward(self, user, item):
        # User and item embeddings
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)
        
        # User and item biases
        user_bias = self.user_bias(user).squeeze()  # (batch_size,)
        
        # Inner product of embeddings
        interaction = (user_embed * item_embed).sum(1)  # (batch_size,)
        
        # Final prediction with biases
        prediction = interaction + user_bias 
        return prediction

In [31]:
num_users = len(data['label2idx']['user_id'])
num_items = len(data['label2idx']['isbn'])
embedding_dim = 16  # 임베딩 차원 크기, 필요에 따라 조정

model = MatrixFactorization(num_users, num_items, embedding_dim)

In [32]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 25  # 필요한 만큼 설정

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for user, item, rating in train_loader:
        optimizer.zero_grad()
        prediction = model(user, item)
        loss = criterion(prediction, rating)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}")

Epoch 1, Training Loss: 12.77199574251895
Epoch 2, Training Loss: 5.867089594857713
Epoch 3, Training Loss: 4.67196586861131
Epoch 4, Training Loss: 4.452914923889517
Epoch 5, Training Loss: 3.9122881164737775
Epoch 6, Training Loss: 3.6800781438685877
Epoch 7, Training Loss: 3.446075886574993
Epoch 8, Training Loss: 3.281334041952341
Epoch 9, Training Loss: 3.155237391982717
Epoch 10, Training Loss: 3.0241582537373755
Epoch 11, Training Loss: 2.945221598714304
Epoch 12, Training Loss: 2.847526587070303
Epoch 13, Training Loss: 2.7915063499409705
Epoch 14, Training Loss: 2.6966823228309686
Epoch 15, Training Loss: 2.662027607324176
Epoch 16, Training Loss: 2.5969815198759463
Epoch 17, Training Loss: 2.5730092145275663
Epoch 18, Training Loss: 2.511145027170193
Epoch 19, Training Loss: 2.49093701208339
Epoch 20, Training Loss: 2.439613934079458
Epoch 21, Training Loss: 2.421778993975584
Epoch 22, Training Loss: 2.3718247465357267
Epoch 23, Training Loss: 2.3687169763213354
Epoch 24, Tra

In [34]:
# X_test에 rating 열을 추가
X_test['rating'] = 0

In [35]:
# test 데이터에 대한 예측값 생성 (sample_submission.csv 사용)
test_dataset = RatingsDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
predictions = []
for user, item, _ in test_loader:
    prediction = model(user, item)
    predictions.extend(prediction.tolist())


In [38]:
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [39]:
# sample_submission에 rating 열을 예측값으로 대체
sample_submission['rating'] = predictions + X_test_mean

In [41]:
sample_submission.to_csv('./data/submission.csv', index=False)